El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Introducción


El objetivo de este proyecto probar diferentes métodos de potenciación de gradiente para compararlos con el modelo de regresión lineal y bosque aleatorio para evaluar su calidad para predecir el valor de mercado de los autos con base en diferentes características del auto como la marca, tipo de vehiculo, transmisión, si ha sido reparado, etc. Ademas de evaluar los tiempos de entrenamiento y predicción de los modelos.

## Preparación de datos

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

### Carga de datos

In [2]:
df = pd.read_csv('/datasets/car_data.csv')

In [3]:
#Usar método info para evaluar los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df.sample(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
304830,29/03/2016 14:47,350,wagon,1998,auto,131,mondeo,150000,7,petrol,ford,no,29/03/2016 00:00,0,12681,31/03/2016 07:45
233124,23/03/2016 14:52,4950,sedan,1997,manual,75,golf,40000,7,petrol,volkswagen,no,23/03/2016 00:00,0,87778,05/04/2016 17:21
120678,01/04/2016 20:39,3800,NaN,2017,manual,115,corsa,150000,0,NaN,opel,no,01/04/2016 00:00,0,94269,05/04/2016 17:26
117321,25/03/2016 08:53,550,small,1996,manual,0,1_reihe,150000,7,petrol,peugeot,no,25/03/2016 00:00,0,44141,28/03/2016 07:45
180819,02/04/2016 16:59,750,small,1996,manual,50,fiesta,125000,2,petrol,ford,no,02/04/2016 00:00,0,28779,02/04/2016 16:59
279231,10/03/2016 01:54,2950,NaN,2017,auto,0,NaN,150000,6,NaN,opel,NaN,09/03/2016 00:00,0,13599,11/03/2016 19:44
178950,15/03/2016 15:58,400,bus,1997,manual,0,galaxy,150000,7,petrol,ford,no,15/03/2016 00:00,0,53879,15/03/2016 16:39
126530,16/03/2016 11:50,2400,wagon,2006,NaN,0,sharan,150000,11,gasoline,volkswagen,no,16/03/2016 00:00,0,86368,16/03/2016 11:50
208353,27/03/2016 15:58,3999,NaN,2017,manual,0,vectra,150000,9,NaN,opel,NaN,27/03/2016 00:00,0,79341,27/03/2016 15:58
168073,01/04/2016 00:56,11800,suv,2011,manual,101,tiguan,150000,7,gasoline,volkswagen,no,01/04/2016 00:00,0,1069,07/04/2016 05:15


In [5]:
#Aplicar método describe para evaluar los datos númericos.
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
#Aplicar método nunique para revisar la cantidad de valores unicos de las columnas.
df.nunique()

DateCrawled          15470
Price                 3731
VehicleType              8
RegistrationYear       151
Gearbox                  2
Power                  712
Model                  250
Mileage                 13
RegistrationMonth       13
FuelType                 7
Brand                   40
NotRepaired              2
DateCreated            109
NumberOfPictures         1
PostalCode            8143
LastSeen             18592
dtype: int64

Durante la exploración inicial de datos no se encontraron valores ausentes en las columnas de datos numéricos. En las columnas de datos categericos si hay valores ausentes

In [7]:
df.fillna('None', inplace = True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        354369 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            354369 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              354369 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           354369 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        354369 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

## Entrenamiento del modelo 

In [9]:
#Definir las columnas para features y target

#No se considera la columna Model para las caracteristicas ya que tiene demasiados valores unicos y dificulta el entrenamiento del modelo
features = ['VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Mileage', 'FuelType', 'Brand', 'NotRepaired'] 
target = ['Price']

In [10]:
#Asignar las caracteristicas a las variables df_features y df_target
df_features = df[features]
df_target = df[target]

In [11]:
df_features.head()

,VehicleType,RegistrationYear,Gearbox,Power,Mileage,FuelType,Brand,NotRepaired
0,None,1993,manual,0,150000,petrol,volkswagen,None
1,coupe,2011,manual,190,125000,gasoline,audi,yes
2,suv,2004,auto,163,125000,gasoline,jeep,None
3,small,2001,manual,75,150000,petrol,volkswagen,no
4,small,2008,manual,69,90000,gasoline,skoda,no


In [12]:
df_target.head()

,Price
0,480
1,18300
2,9800
3,1500
4,3600


In [13]:
#Aplicar OHE a features

df_features_ohe = pd.get_dummies(df_features, drop_first=True)

In [14]:
#Verificar si se realizo correctamente OHE
df_features_ohe.head()

,RegistrationYear,Power,Mileage,VehicleType_bus,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
0,1993,0,150000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2011,190,125000,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2004,163,125000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2001,75,150000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
4,2008,69,90000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
#Dividir en conjunto de entrenamiento y prueba en proporción 80:20 a datos con OHE

features_train_ohe, features_test_ohe, target_train_ohe, target_test_ohe = train_test_split(df_features_ohe, df_target, test_size=0.20, random_state=12345)

### Modelo de regresión lineal

In [16]:
%%time
#Entrenar modelo de regresión lineal y hacer predicciones para conjunto de prueba
model_rl = LinearRegression()
model_rl.fit(features_train_ohe, target_train_ohe)


CPU times: user 2.4 s, sys: 429 ms, total: 2.83 s
Wall time: 2.83 s


LinearRegression()

In [17]:
%%time
predictions_rl_test = model_rl.predict(features_test_ohe)

CPU times: user 45 ms, sys: 25.2 ms, total: 70.2 ms
Wall time: 101 ms


In [18]:
#Caclular RECM para modelo de regresión lineal

RECM_rl = mean_squared_error(target_test_ohe, predictions_rl_test)**0.5
print("RECM del modelo de regresión lineal en el conjunto de prueba:", RECM_rl)

RECM del modelo de regresión lineal en el conjunto de prueba: 3348.0400620189225


### Modelo de bosque aleatorio

In [19]:
%%time
#Entrenar modelo de bosque aleatorio
model_rfr = RandomForestRegressor(max_depth=15, n_estimators=10, random_state=12345)
model_rfr.fit(features_train_ohe, target_train_ohe.values.ravel())
predictions_rfr_test = model_rfr.predict(features_test_ohe)

CPU times: user 19 s, sys: 59.1 ms, total: 19 s
Wall time: 19.1 s


In [20]:
%%time
# hacer predicciones para conjunto de prueba
predictions_rfr_test = model_rfr.predict(features_test_ohe)

CPU times: user 171 ms, sys: 12 ms, total: 183 ms
Wall time: 189 ms


In [21]:
#Caclular RECM para modelo de bosque aleatorio
RECM_rfr = mean_squared_error(target_test_ohe, predictions_rfr_test)**0.5
print("RECM del modelo de bosque aleatorio en el conjunto de prueba:", RECM_rfr)

RECM del modelo de bosque aleatorio en el conjunto de prueba: 1868.3904882340232


### Modelo LightGBM

In [22]:
#Dividir en conjunto de entrenamiento y validación en proporción 80:20

x_train, x_valid, y_train, y_valid = train_test_split(df_features, df_target, test_size=0.20, random_state=12345)


In [23]:
#Definir variables categoricas
cat_features = ['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'NotRepaired']

In [24]:
#Cambiar tipo de datos de conjuntos de entrenamiento y validación a category
for c in cat_features:
    x_train[c] = x_train[c].astype('category')
    x_valid[c] = x_valid[c].astype('category')

/tmp/ipykernel_209/3306839157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[c] = x_train[c].astype('category')
/tmp/ipykernel_209/3306839157.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_valid[c] = x_valid[c].astype('category')


In [25]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283495 entries, 50353 to 217570
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   VehicleType       283495 non-null  category
 1   RegistrationYear  283495 non-null  int64   
 2   Gearbox           283495 non-null  category
 3   Power             283495 non-null  int64   
 4   Mileage           283495 non-null  int64   
 5   FuelType          283495 non-null  category
 6   Brand             283495 non-null  category
 7   NotRepaired       283495 non-null  category
dtypes: category(5), int64(3)
memory usage: 10.0 MB


In [26]:
#Cambiar conjuntos a lgb datasets
train_data=lgb.Dataset(x_train, label=y_train, feature_name=features, categorical_feature=cat_features, free_raw_data=False)
valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

In [27]:
#Definir parametros para Light GBM
param_lgbm = {'num_leaves': 150, 'max_depth':10, 'learning_rate':.15, 'max_bin':200, 'boosting_type': 'gbdt', 'metric': 'rmse'}

In [28]:
%%time
#Entrenamiento del modelo
num_round=50
model_lgbm=lgb.train(param_lgbm,train_data,num_round)


/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003609 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 378
[LightGBM] [Info] Number of data points in the train set: 283495, number of used features: 8
[LightGBM] [Info] Start training from score 4414.416558
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
CPU times: user 3.74 s, sys: 24.6 ms, total: 3.77 s
Wall time: 3.78 s


In [29]:
%%time
#Realizar predicciones del conjunto de validación
y_pred_lgmb = model_lgbm.predict(x_valid, num_iteration=model_lgbm.best_iteration)


CPU times: user 647 ms, sys: 6.97 ms, total: 654 ms
Wall time: 709 ms


In [30]:
#Caclular RECM para modelo Light GBM
RECM_lgbm = mean_squared_error(y_valid, y_pred_lgmb)**0.5
print("RECM del modelo de bosque aleatorio en el conjunto de validación:", RECM_lgbm)

RECM del modelo de bosque aleatorio en el conjunto de validación: 1790.3625584271483


### Modelo CatBoost

In [31]:
#Dividir en conjunto de entrenamiento y validación en proporción 80:20 con del df original

x_train_1, x_valid_1, y_train_1, y_valid_1 = train_test_split(df_features, df_target, test_size=0.20, random_state=12345)


In [32]:
%%time

#Llamar al catboosregressor
model_catboost = CatBoostRegressor(cat_features = cat_features, max_depth=10, learning_rate=0.5, n_estimators=150, random_seed=12345, simple_ctr='Counter')
#Entrenar modelo con conjunto de entrenamiento
model_catboost.fit(x_train_1, y_train_1)

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


0:	learn: 3148.4035591	total: 240ms	remaining: 35.8s
1:	learn: 2605.1367805	total: 411ms	remaining: 30.4s
2:	learn: 2383.3293435	total: 589ms	remaining: 28.9s
3:	learn: 2252.3657393	total: 779ms	remaining: 28.4s
4:	learn: 2134.0746443	total: 945ms	remaining: 27.4s
5:	learn: 2089.3221764	total: 1.12s	remaining: 26.8s
6:	learn: 2046.0246386	total: 1.3s	remaining: 26.7s
7:	learn: 2027.4780198	total: 1.48s	remaining: 26.3s
8:	learn: 2008.0253952	total: 1.66s	remaining: 26s
9:	learn: 1995.9446138	total: 1.83s	remaining: 25.6s
10:	learn: 1988.7138146	total: 2s	remaining: 25.3s
11:	learn: 1973.0268726	total: 2.17s	remaining: 25s
12:	learn: 1961.0616676	total: 2.35s	remaining: 24.8s
13:	learn: 1948.5194363	total: 2.52s	remaining: 24.5s
14:	learn: 1944.1118186	total: 2.69s	remaining: 24.2s
15:	learn: 1937.4520014	total: 2.86s	remaining: 24s
16:	learn: 1927.6947541	total: 3.04s	remaining: 23.8s
17:	learn: 1922.4549610	total: 3.19s	remaining: 23.4s
18:	learn: 1913.2682548	total: 3.35s	remaining: 

In [33]:
%%time
#Realizar predicciones del conjunto de validación
y_pred_catboost = model_catboost.predict(x_valid_1)

CPU times: user 127 ms, sys: 3.98 ms, total: 131 ms
Wall time: 137 ms


In [34]:
#Caclular RECM para modelo Catboost
RECM_catboost = mean_squared_error(y_valid_1, y_pred_catboost)**0.5
print("RECM del modelo de bosque aleatorio en el conjunto de validación:", RECM_catboost)

RECM del modelo de bosque aleatorio en el conjunto de validación: 1826.5782027738


## Análisis del modelo

Comparación de calidad y tiempo de ejecución de entrenamiento de los diferentes modelos:

Regresión lineal =
    RECM: 3348.04 / tiempo: 2.83s
   
Bosque aleatorio =
    RECM: 1868.39 / tiempo: 19.1s
    
LightGBM =
    RECM: 1790.36 / tiempo: 3.78s

Catboost =
    RECM: 1826.57 / tiempo: 26.7s
    
La velocidad para la predicción de todos los modelos esta por debajo de 1 segundo.

Con base en el resultado de RECM y el tiempo de entrenamiento se puede observar que el modelo que tiene los mejores resultados es LightGBM con un tiempo menor a 4 segundos y el menor RECM.

Todos los modelos tienen un mejor desempeño que la prueba de cordura (modelo de regresión lineal).

## Conclusión



Al realizar este proyecto en el que se compararon los modelos de regresión lineal, y bosque aleatorio con los métodos de potenciación de gradiente, se concluye que todos los modelos tienen una mejor calidad que el de Regresión lineal ya que en este no hay ajuste de hiperparametros. Si existe una diferencia significativa entre el RECM del resto de los modelos contra el de regresión lineal.
El mejor modelo para este proyecto fue el de potenciación de gradiente LightGBM, ya que se obtuvo el menor RECM y un tiempo de entrenamiento muy bajo.
Con al ajuste de hiperparametros se podria mejorar la calidad del modelo aun mas, pero esto tiene una repercusión en el tiempo necesario para realizar el entrenamiento.